# Decorators
We saw in the previous chapter that functions are objects, and as such, can be manipulated just like any other variable in our programs. We can, for example, pass a function to a function. But let's rewind, and start with a basic function. 

## 1. Function features

In [1]:
#returns True if a number is even
def iseven(number):
    return number % 2 == 0

#returns True if a number is odd
def isodd(number):
    return number % 2 != 0

print(iseven(10))
print(isodd(10))

True
False


Like many other objects, functions have properties. For instance, the `__name__` property returns the name of the function.

In [2]:
print(iseven.__name__)
print(isodd.__name__)

iseven
isodd


So let's write a function - called `log` - that: 
- accepts as only argument another function (for example, `iseven` or `isodd`)
- opens a file called `history.txt`
- writes the current time and name of function to the file
- closes the file
- finally returns the passed function

In [3]:
import datetime

def log(function, mode="a"):
    with open('history.txt', mode) as file: 
        #create a string with the current time
        now = datetime.datetime.now().strftime("%H:%M:%S:%f")
        
        #write the current time and function
        file.write(now + ", Logging " + function.__name__ + '\n')
    
    #return the original function
    return function

#Let's call our log function
log(iseven, "w")
log(isodd, "a")

#Let's read what the function.txt file now contains: 
with open('history.txt', 'r') as file: 
    print(file.read())

22:36:31:098278, Logging iseven
22:36:31:099429, Logging isodd



## 2. Let's do some magic: our first decorator
Lets do some magic here: let's create a function called `wrapper`, which: 
1. takes another function as single argument
2. defines an inner function, which takes the same arguments as the argument function 
3. the inner function can execute any arbitrary code, including calling the argument function
4. the wrapper function then returns the inner function

In other words: 

```python
def wrapper(function):
    def inner(*args, **kwargs): 
        #do something here, for instance log a function call to a file
        log(function)
        return function(*args, **kwargs)
    return inner
```

In [4]:
#let's see this in practice
def wrapper(function):
    def inner(number):
        #we log the function call to the file
        log(function)
        return function(number)
    return inner

In [5]:
#f is the wrapped iseven function
f = wrapper(iseven)

#let's call
print(f(10))

#Let's read what the hisory.txt file now contains: 
#You'll see that there is now a third line
with open('history.txt', 'r') as file: 
    print(file.read())

True
22:36:31:098278, Logging iseven
22:36:31:099429, Logging isodd
22:36:31:166264, Logging iseven



In [6]:
#what happens if we assign the result of wrapper(isodd) back to isodd
isodd = wrapper(isodd)

print(isodd(20))
print(isodd(21))

with open('history.txt', 'r') as file: 
    print(file.read())

False
True
22:36:31:098278, Logging iseven
22:36:31:099429, Logging isodd
22:36:31:166264, Logging iseven
22:36:31:192996, Logging isodd
22:36:31:193852, Logging isodd



## 3. Syntactic sugar

In [7]:
#use the below syntax to create a decorator

def log(function, mode="a"):
    with open('history.txt', mode) as file: 
        #create a string with the current time
        now = datetime.datetime.now().strftime("%H:%M:%S:%f")
        
        #write the current time and function
        file.write(now + ", Logging " + function.__name__ + '\n')
    
    #return the original function
    return function

@log
def isprime(n):
    for i in range(2, n):
        if n % i == 0: 
            return False
    return True

print(isprime(20))

False


In [8]:
with open('history.txt', 'r') as file: 
    print(file.read())

22:36:31:098278, Logging iseven
22:36:31:099429, Logging isodd
22:36:31:166264, Logging iseven
22:36:31:192996, Logging isodd
22:36:31:193852, Logging isodd
22:36:31:243198, Logging isprime



In [9]:
#the above is equivalent to 
def wrapper(function): 
    def log(*args, **kwargs):
        with open('history.txt', 'a') as file: 
            #create a string with the current time
            now = datetime.datetime.now().strftime("%H:%M:%S:%f")

            #write the current time and function
            file.write(now + ", Logging " + function.__name__ + '\n')
        return function(*args, **kwargs)
    return log

def isprime(n):
    for i in range(2, n):
        if n % i == 0: 
            return False
    return True

isprime = wrapper(isprime)

In [10]:
print(isprime(23))

with open('history.txt', 'r') as file: 
    print(file.read())

True
22:36:31:098278, Logging iseven
22:36:31:099429, Logging isodd
22:36:31:166264, Logging iseven
22:36:31:192996, Logging isodd
22:36:31:193852, Logging isodd
22:36:31:243198, Logging isprime
22:36:31:320128, Logging isprime



In [11]:
#another example
def ispalindromic(number):
    return str(number) == str(number)[::-1]

def throttle(function):
    def inner(*args, **kwargs):
        #record the current time
        now = datetime.datetime.now()

        #wait for 5 seconds
        while datetime.datetime.now() < (now + datetime.timedelta(seconds=5)):
            continue
            
        return function(*args, **kwargs)
    return inner

#first test it directly
print(ispalindromic(1991))

#now decorate
ispalindromic = throttle(ispalindromic)
print(ispalindromic(1991))

True
True


In [12]:
@throttle
def ispalindromic(number):
    return str(number) == str(number)[::-1]

print(ispalindromic(123454312))

False
